In [1]:
using DataFrames, CSV, Query #Data handling
using Convex, GLPKMathProgInterface # Optimization tools

fn = "MOWOG1entries.csv"
c1_list=["AS", "BS", "CS", "DS", "ES", "FS", "GS", "HS","SS","SSR"] # Combined 1 classes
_run_groups = 2 #Number of run groups for the event
_max_to_bump = 4 #Maximum number of entrants in the class that will still be bumped to combined

#Distribute an integer over N integer parts
function distribute_int(a::T,n::T) where {T<:Integer}
    (num,den) = divrem(a,n)
    [ifelse(i<=den,num+1,num) for i=1:n]
end
@assert distribute_int(10,3)==[4,3,3]

Randomly assign 23 entrants an exempt work position for testing purposes
with a preset random seed

In [2]:
df=CSV.read(fn); #Read the CSV to a DataFrame
rename!(df, Symbol("Modifier/PAX") => :Index)
#rename!(df, Symbol("First Name")=> :FirstName)
#rename!(df, Symbol("Last Name")=> :LastName)
delete!(df, [Symbol("Segment Name"),:Group])
df[:IndexClass]=map(x-> ismissing(x[:Index]) ? x[:Class] |> String : x[:Index] |> String, eachrow(df))
head(df)

,Class,Year,Make,Model,Index,IndexClass
1,GS,2013,Ford,Focus ST,Z,Z
2,HS,2015,Ford,Fiesta ST,Z,Z
3,ES,2004,Toyota,MR2,P,P
4,BS,1992,Chevrolet,Corvette,missing,BS
5,STS,1988,Honda,CRX,P,P
6,N,1996,Lexus,sc400,missing,N


In [3]:
n_drivers=nrow(df);
exempt_drivers=fill(false,n_drivers);
srand(562161);
exempt_drivers[randperm(n_drivers)[1:23]]=true;
df[:Exempt]=exempt_drivers;
head(df)

,Class,Year,Make,Model,Index,IndexClass,Exempt
1,GS,2013,Ford,Focus ST,Z,Z,false
2,HS,2015,Ford,Fiesta ST,Z,Z,false
3,ES,2004,Toyota,MR2,P,P,true
4,BS,1992,Chevrolet,Corvette,missing,BS,false
5,STS,1988,Honda,CRX,P,P,false
6,N,1996,Lexus,sc400,missing,N,false


In [4]:
#Count up the drivers per class
df=@from i in df begin
    @group i by i.IndexClass into g
    @select {Class=g.key, Drivers=length(g),Exempt=sum(g..Exempt)}
    @collect DataFrame
end
head(df)

,Class,Drivers,Exempt
1,Z,30,5
2,P,21,6
3,BS,7,1
4,N,29,1
5,FS,1,0
6,ASP,1,0


In [5]:
let d=df[find(x->x[:Class]=="N",eachrow(df)),:]
    DataFrame(Class=(@. "N"*string(1:_run_groups)),
        Drivers=distribute_int(d[:Drivers][1],_run_groups),
        Exempt=distribute_int(d[:Exempt][1],_run_groups)) |> x -> append!(df,x)
end

df=@from i in df begin
    @where i.Class != "N"
    @select i
    @collect DataFrame
end
df[:Workers]=df[:Drivers].-df[:Exempt]
head(df)

,Class,Drivers,Exempt,Workers
1,Z,30,5,25
2,P,21,6,15
3,BS,7,1,6
4,FS,1,0,1
5,ASP,1,0,1
6,SSC,2,0,2


In [6]:
# Create our variables
N = nrow(df)
x = Variable((N,_run_groups), :Bin) #Class allocation variable

Variable of
size: (26, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [7]:
#Each class must be in exactly 1 run group
constr=sum(x,2).==1;

In [8]:
#split novice subgroups, only one novice group per run group
constr+=let ind=map(x->x[1]=='N',df[:Class]) |> find
    [sum(x[ind,:],1).==1]
end;

In [9]:
#keep Combined classes together if necessary
constr+=let ind=[any(d[:Class].==c1_list) && d[:Drivers] <= _max_to_bump for d in eachrow(df)] |> find
    "Combining $(join(df[ind,:Class],',')) due to <= $_max_to_bump drivers" |> println
    [x[ind[1:end-1],run_group].==x[ind[2:end],run_group] for run_group=1:_run_groups]
end
constr+=let ind=[!any(d[:Class].==c1_list) && d[:Drivers] <= _max_to_bump for d in eachrow(df)] |> find
    "Combining $(join(df[ind,:Class],',')) due to <= $_max_to_bump drivers" |> println
    [x[ind[1:end-1],run_group].==x[ind[2:end],run_group] for run_group=1:_run_groups]
end;

Combining FS,HS,GS,DS,CS due to <= 4 drivers
Combining ASP,SSC,STX,STU,DM,EM,SSM,V,CAM-S,STH,CAM-T,X,STS,SSP due to <= 4 drivers


In [10]:
#Split Pro & Z
constr+=let ind=[any(d[:Class].==["P","Z"]) for d in eachrow(df)] |> find
    sum(x[ind,:],1).<=1
end;

In [11]:
#Expressions that can be used in the optimizer
rungroup_workers=sum(x.*df[:Workers],1) |> vec #Workers available per run group
rungroup_drivers=sum(x.*df[:Drivers],1) |> vec #Drivers in each run group

AbstractExpr with
head: reshape
size: (2, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [12]:
# Define the problem's optimization, under required constraints
p=maximize(minimum(rungroup_workers),constr);  #Maximize, the Minimum # of workers in a run group

In [13]:
solve!(p, GLPKSolverMIP()); # Ignore the deprecation warning

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Int64}, ::Int64) at .\deprecated.jl:57
 [3] conic_form!(::Convex.IndexAtom, ::Convex.UniqueConicForms) at C:\Users\dberge\.julia\v0.6\Convex\src\atoms/affine\index.jl:57
 [4] conic_form!(::Convex.MultiplyAtom, ::Convex.UniqueConicForms) at C:\Users\dberge\.julia\v0.6\Convex\src\atoms/affine\multiply_divide.jl:85
 [5] conic_form!(::Convex.AdditionAtom, ::Convex.UniqueConicForms) at C:\Users\dberge\.julia\v0.6\Convex\src\atoms/affine\add_subtract.jl:108
 [6] conic_form!(::Convex.EqConstraint, ::Convex.UniqueConicForms) at C:\Users\dberge\.julia\v0.6\Convex\src\constraints\constraints.jl:43
 [7] conic_form!(::Convex.Problem, ::Convex.UniqueConicForms) at C:\Users\dberge\.julia\v0.6\Convex\src\problems.jl:101
 [8] conic_problem(::Convex.Problem) at C:\Users\dberge\.julia\v0.6\Convex\src\problems.jl:124
 [9] #solve!#25(::Bool, ::Bool, ::Bool, ::Function, ::Convex.Problem) at C:\Users\dberge\.julia\v0.6\Conv

In [14]:
#What is the status of the solutioin
p.status

:Optimal

In [15]:
let drivers=evaluate(rungroup_drivers)
    for i=1:length(drivers)
        println("Run group #$i has $(Integer(drivers[i])) drivers")
    end
end

Run group #1 has 67 drivers
Run group #2 has 72 drivers


In [16]:
let drivers=evaluate(rungroup_workers)
    for i=1:length(drivers)
        println("Run group #$i has $(Integer(drivers[i])) workers")
    end
end

Run group #1 has 59 workers
Run group #2 has 57 workers


In [17]:
let drivers=evaluate(rungroup_drivers .- rungroup_workers)
    for i=1:length(drivers)
        println("Run group #$i has $(Integer(drivers[i])) exempt workers")
    end
end

Run group #1 has 8 exempt workers
Run group #2 has 15 exempt workers


In [18]:
println("Run group #1:")
println.(df[:Class][evaluate(x)[:,1].>0.0]);

Run group #1:
Z
FS
HS
ES
GS
DS
SMF
CS
N2


In [19]:
println("Run Group #2:")
println.(df[:Class][evaluate(x)[:,2].>0.0]);

Run Group #2:
P
BS
ASP
SSC
STX
STU
DM
EM
SSM
V
CAM-S
STH
CAM-T
X
STS
SSP
N1
